In [9]:
import tensorflow as tf
from tensorflow import keras as k
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas
import numpy as np

In [10]:
model_save = tf.keras.callbacks.ModelCheckpoint('./best_weights.h5',
                             save_best_only = True,
                             save_weights_only = True,
                             monitor = 'val_loss',
                             mode = 'min', verbose = 1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0.001,
                           patience = 10, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3,
                              patience = 2, min_delta = 0.001,
                              mode = 'min', verbose = 1)

In [11]:
data = pandas.read_csv('/Users/polskafly/PycharmProjects/music_identifier/Data/features_3_sec.csv')
data.head()


,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [12]:
data = data.drop(labels='filename', axis=1)

In [13]:
genres = data.iloc[:, -1]
encoder = LabelEncoder()

y = encoder.fit_transform(genres)

data.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [14]:
data.head()

scaler = StandardScaler()
x = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [15]:
features_train, features_test, labels_train, labels_test = train_test_split(x, y, test_size=0.33)

In [37]:
model = k.Sequential([
    k.layers.Dense(512, activation='relu', input_shape=(features_train.shape[1],)),
    k.layers.Dropout(0.2),

    k.layers.Dense(256, activation='relu'),
    k.layers.Dropout(0.2),

    k.layers.Dense(128, activation='relu'),
    k.layers.Dropout(0.2),

    k.layers.Dense(64, activation='relu'),
    k.layers.Dropout(0.2),

    k.layers.Dense(10, activation='softmax'),
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [38]:
classifier = model.fit(features_train,
                    labels_train,
                    epochs=180,
                    batch_size=128,
                    validation_data=(features_test, labels_test),
                    callbacks=[model_save, early_stop, reduce_lr])


Epoch 1/180
53/53 [==============================] - 1s 7ms/step - loss: 1.9162 - accuracy: 0.3148 - val_loss: 1.1642 - val_accuracy: 0.5957

Epoch 00001: val_loss improved from inf to 1.16417, saving model to ./best_weights.h5
Epoch 2/180
53/53 [==============================] - 0s 6ms/step - loss: 1.2094 - accuracy: 0.5864 - val_loss: 0.9059 - val_accuracy: 0.7015

Epoch 00002: val_loss improved from 1.16417 to 0.90592, saving model to ./best_weights.h5
Epoch 3/180
53/53 [==============================] - 0s 4ms/step - loss: 0.9868 - accuracy: 0.6641 - val_loss: 0.7882 - val_accuracy: 0.7358

Epoch 00003: val_loss improved from 0.90592 to 0.78821, saving model to ./best_weights.h5
Epoch 4/180
53/53 [==============================] - 0s 4ms/step - loss: 0.8222 - accuracy: 0.7196 - val_loss: 0.7365 - val_accuracy: 0.7534

Epoch 00004: val_loss improved from 0.78821 to 0.73648, saving model to ./best_weights.h5
Epoch 5/180
53/53 [==============================] - 0s 4ms/step - loss: 0.7

53/53 [==============================] - 0s 3ms/step - loss: 0.1016 - accuracy: 0.9671 - val_loss: 0.3565 - val_accuracy: 0.9060

Epoch 00037: val_loss did not improve from 0.35622
Epoch 38/180
53/53 [==============================] - 0s 5ms/step - loss: 0.1069 - accuracy: 0.9643 - val_loss: 0.3565 - val_accuracy: 0.9063

Epoch 00038: val_loss did not improve from 0.35622

Epoch 00038: ReduceLROnPlateau reducing learning rate to 2.1870000637136398e-07.
Epoch 39/180
53/53 [==============================] - 0s 4ms/step - loss: 0.1180 - accuracy: 0.9596 - val_loss: 0.3565 - val_accuracy: 0.9063

Epoch 00039: val_loss did not improve from 0.35622
Epoch 40/180
53/53 [==============================] - 0s 3ms/step - loss: 0.1093 - accuracy: 0.9666 - val_loss: 0.3565 - val_accuracy: 0.9063

Epoch 00040: val_loss did not improve from 0.35622
Restoring model weights from the end of the best epoch.

Epoch 00040: ReduceLROnPlateau reducing learning rate to 6.561000276406048e-08.
Epoch 00040: early

In [39]:
test_loss, test_acc  = model.evaluate(features_test, labels_test, batch_size=128)

26/26 [==============================] - 0s 1ms/step - loss: 0.3563 - accuracy: 0.9045
